# Calculating all heuristics for a given tsp dataset

In [1]:
import heuristics
import fileops as fo
import networkx as nx
import numpy as np
from networkx.algorithms.approximation import christofides, greedy_tsp
from IPython.display import clear_output
import threading
import time


In [2]:
def execute_mst(heuristic_name, function, instance, data, path):
    print(f'    COMPUTING:   {heuristic_name}')
    duration = time.time()
    mst, tree = function(instance)
    mst = int(mst)
    duration = time.time() - duration
    print(f'    DONE:        {heuristic_name} : {mst}   ---   TIME: {duration}')
    data.append(mst)
    fo.save_as_json(data, f'tspset2_heuristics/{path}.json')
    return tree


def execute_tree_heuristic(heuristic_name, function, instance, tree, data, path):
    print(f'    COMPUTING:   {heuristic_name}   ---   ')
    duration = time.time()
    result = -1
    try:
        result = function(instance, tree)
        result = int(result)
    except:
        print('An error with a tree based heuristic occured')
        result = -1
    duration = time.time() - duration
    print(f'    DONE:        {heuristic_name} : {result}   ---   TIME: {duration}')
    data.append(result)
    fo.save_as_json(data, f'tspset2_heuristics/{path}.json')
    
def exectue_heuristic(heuristic_name, function, instance, data, path):
    print(f'    COMPUTING:   {heuristic_name}   ---   ')
    duration = time.time()
    result = function(instance)
    result = int(result)
    duration = time.time() - duration
    print(f'    DONE:        {heuristic_name} : {result}   ---   TIME: {duration}')
    data.append(result)
    fo.save_as_json(data, f'tspset2_heuristics/{path}.json')

def update_heu(input: dict):
    for key in input.keys():
        input[key] = fo.load_from_json(f'tspset2_heuristics/{key}.json')
    return input
        

In [ ]:
instances = 2000
heu = {
    #'mst':[],
    'christo': [],
    #'onetree': [],
    'fi': [],
    #'greedy': [],
    #'mstheu': [],
    'ni': [],
    #'nn': [],
    #'opt': [],
    'ri': []
}
for i in range(instances):
    instance = fo.load_from_json(f'tspset2/ctsp_{i}_sol.json')
    dimension = instance['comment']
    print(f'PROCESSING INSTANCE {i+1} OF {instances} PROGRESS: {((i+1)/instances)*100}% \nINSTANCE SIZE: {dimension}')
    
    # first we need the mst  
    #tree           =                                            execute_mst('MINIMUM SPANNING TREE          ', heuristics.mst, instance, heu['mst'], 'mst')
    tree = None
    # in multithreading we comute mst dependend heuristics
    christo_thread = threading.Thread( target=execute_tree_heuristic, args=('CHRISTOFIDES                   ', heuristics.christo, instance, tree, heu['christo'], 'christo') )
    #onetree_thread = threading.Thread( target=execute_tree_heuristic, args=('1-TREE                         ', heuristics.onetree, instance, tree, heu['onetree'], 'onetree') )
    #mstheu_thread  = threading.Thread( target=execute_tree_heuristic, args=('MINIMUM SPANNING TREE HEURISTIC', heuristics.mstheu, instance, tree, heu['mstheu'], 'mstheu') )
    
    # and all other heuristics
    fi_thread      = threading.Thread( target=exectue_heuristic,      args=('FARTHEST INSERTION             ', heuristics.fi, instance,  heu['fi'], 'fi') )
    #greedy_thread  = threading.Thread( target=exectue_heuristic,      args=('GREEDY HEURISTIC               ', heuristics.greedy, instance,  heu['greedy'], 'greedy') )
    ni_thread      = threading.Thread( target=exectue_heuristic,      args=('NEAREST INSERTION              ', heuristics.ni, instance,  heu['ni'], 'ni') )
    #nn_thread      = threading.Thread( target=exectue_heuristic,      args=('NEAREST NEIGHBOR               ', heuristics.nn, instance,  heu['nn'], 'nn') )
    #opt_thread     = threading.Thread( target=exectue_heuristic,      args=('OPTIMAL TOUR                   ', heuristics.opt, instance,  heu['opt'], 'opt') )
    ri_thread      = threading.Thread( target=exectue_heuristic,      args=('RANDOM INSERTION               ', heuristics.ri, instance,  heu['ri'], 'ri') )

    
    threads = [
        fi_thread, 
        ni_thread, 
        ri_thread,
        christo_thread 
        #onetree_thread, 
        #mstheu_thread, 
        #greedy_thread, 
        #nn_thread, 
        #opt_thread
    ]
    
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()
        
    heu = update_heu(heu)
    if i%2 != 0:
        clear_output()
   

PROCESSING INSTANCE 17 OF 2000 PROGRESS: 0.8500000000000001% 
INSTANCE SIZE: randomly generated clustered problem with 495 cities
    COMPUTING:   FARTHEST INSERTION                ---   
    COMPUTING:   NEAREST INSERTION                 ---   
    COMPUTING:   RANDOM INSERTION                  ---   
    COMPUTING:   CHRISTOFIDES                    : ...   
    DONE:        CHRISTOFIDES                    : 71236   ---   TIME: 33.080448627471924
    DONE:        RANDOM INSERTION                : 68213   ---   TIME: 118.28363513946533


In [7]:
instance = fo.load_from_json('tspset2/ctsp_0_sol.json')